In [18]:
# Read in data to dataframe
import pandas as pd
from pandas import read_pickle
import os


In [19]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized',filename))
    li.append(dataframe)

data = pd.concat(li, axis=0, ignore_index=True)

In [20]:
columns = [ 'HR_mean', 'HR_std', 'HR_min', 'HR_max', 'HRV_RMSSD',
       'HRV_MeanNN', 'HRV_SDNN', 'HRV_CVNN', 'HRV_CVSD', 'HRV_MedianNN',
       'HRV_MadNN', 'HRV_MCVNN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_HF', 'HRV_VHF',
       'HRV_HFn', 'HRV_LFHF', 'HRV_SD1', 'HRV_SD2', 'HRV_SD1SD2', 'RSP_mean',
       'RSP_std', 'RSP_min', 'RSP_max', 'RRV_RMSSD', 'RRV_MeanBB', 'RRV_SDBB',
       'RRV_SD1', 'RRV_SD2', 'RRV_SD2SD1','label','ID']

In [21]:
cw = data[columns]

In [27]:
# Read in data to dataframe
import pandas as pd
from pandas import read_pickle

names = ["cw"]
dataframes = cw

In [28]:
# GAN parameters
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import ParameterSampler

param_dist = {
    "log_frequency": [True, False],
    "generator_lr": loguniform(0.00002, 0.002),
    "generator_decay": loguniform(0.00002, 0.002),
    "discriminator_decay": loguniform(0.00002, 0.002),
    "discriminator_lr": loguniform(0.00002, 0.002),
    "discriminator_decay": loguniform(0.00002, 0.002),
    "discriminator_steps": [1, 5, 10],
}

In [34]:
# function for RandomParameterSearch, given a data set name, dataframe, and parameters to search over.
from sdv.evaluation import evaluate
from sdv.tabular import CTGAN
from time import time


def RandomSearch(df: pd.DataFrame, params: dict, n_iter: int, random_seed=15
) -> dict:

    # generate random sets of parameters sampled over parameter space
    gan_parameter_sampler = ParameterSampler(
        param_distributions=params, n_iter=n_iter, random_state=random_seed
    )
    random_search_param_list = list(gan_parameter_sampler)

    best_parameter_set = {}
    best_score = 0
    count = 1
    # append empty parameter list for default values
    random_search_param_list.append({})

    # loop over all parameter sets and evaluate performance.
    ########################################################
    for parameter_set in random_search_param_list:
        print(
            f'Parameter Search for "{cw}" dataset. Iteration {count} of {N_ITER+1}'
        )
        # print("Parameters:")
        # print("-------------------------------------")
        # for x in parameter_set:
        #     print(x, ": ", parameter_set[x])
        # if not parameter_set:
        #     print("Default parameters.")
        # print("-------------------------------------")
        start_time = time()
        score = 0
        if parameter_set:
            # if the parameter set is not empty, set all attributes from the dicitonary.
            model = CTGAN(
                log_frequency=parameter_set["log_frequency"],
                generator_lr=parameter_set["generator_lr"],
                generator_decay=parameter_set["generator_decay"],
                discriminator_lr=parameter_set["discriminator_lr"],
                discriminator_decay=parameter_set["discriminator_decay"],
                discriminator_steps=parameter_set["discriminator_steps"],
                verbose=False,
            )
        else:
            model = CTGAN()
            parameter_set = {"empty": "using default parameters"}

        # train the model on the given data set
        model.fit(df)
        # generate data equal to the size of the original data set
        new_data = model.sample(num_rows=df.shape[0])
        # evaluate the quality of the new data compared to the old data
        score = evaluate(
            new_data,
            df,
            metrics=[
                "CSTest",
                "KSTest",
                "BNLogLikelihood",
                "GMLogLikelihood",
                "LogisticDetection",
                "SVCDetection",
            ],
            aggregate=True,
        )
        elapsed_time = time() - start_time
        print("Score:", score)
        print(f"Elapsed time: {round(elapsed_time/60, 2)} minutes.")
        print("-------------------------------------\n")
        # save the best score and parameter set
        if score > best_score:
            best_parameter_set = parameter_set
            best_score = score
            model.save(f"GAN_{dataset_name}_model.pkl")
        count += 1
    # print out best parameters
    print("Best Parameters:")
    print("-------------------------------------")
    for x in best_parameter_set:
        print(x, ": ", best_parameter_set[x])
    print("-------------------------------------")
    print("Best Score:", best_score)
    print("-------------------------------------\n")

    # return best parameters
    return best_parameter_set

In [35]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
from pandas.core.common import SettingWithCopyWarning

simplefilter("ignore", category=ConvergenceWarning)
simplefilter("ignore", category=SettingWithCopyWarning)

N_ITER = 150

best_parameters = {}
RandomSearch(cw, params=param_dist, n_iter=N_ITER)

Parameter Search for "      HR_mean    HR_std    HR_min    HR_max  HRV_RMSSD  HRV_MeanNN  HRV_SDNN  \
0    0.000000  0.000000  0.183042  0.000000   0.486688    1.000000  0.126382   
1    0.812426  0.119663  0.457481  1.000000   0.000000    0.180871  0.000000   
2    0.688324  0.406721  0.814940  0.398330   0.260090    0.301102  0.296830   
3    0.303016  1.000000  0.000000  0.876441   1.000000    0.689673  1.000000   
4    0.740267  0.208258  1.000000  0.724301   0.108911    0.250881  0.097553   
..        ...       ...       ...       ...        ...         ...       ...   
156  0.871895  0.183882  1.000000  0.125336   0.280547    0.118401  0.185625   
157  0.550039  1.000000  0.000000  1.000000   1.000000    0.441699  1.000000   
158  0.240803  0.469417  0.326714  0.520218   0.749579    0.742920  0.583076   
159  0.000000  0.244326  0.138062  0.000000   0.651896    1.000000  0.473866   
160  0.232730  0.219379  0.399302  0.249347   0.769476    0.892990  0.451357   

     HRV_CVNN  HR

{'discriminator_decay': 0.0002980602398777751,
 'discriminator_lr': 0.0017901333247018088,
 'discriminator_steps': 10,
 'generator_decay': 0.0004416387260647504,
 'generator_lr': 0.00045743418650565575,
 'log_frequency': False}

In [36]:
best_parameters

{}